In [1]:
import pandas as pd
import datetime
import numpy as np
import mysql.connector
import json
import csv
import os
import matplotlib as plt
import matplotlib.pyplot as pyplot
import dataframe_image as dfi
import requests

### Downloading the most recent sales data directly from the Census Bureau website

In [2]:
# 1. define url path
url = "https://www.census.gov/retail/mrts/www/mrtssales92-present.xlsx"

# 2. download the data behind the URL
response = requests.get(url)

# 3. Open the response into a new file called mrtssales92-present.xlsx
open("mrtssales92-present.xlsx", "wb").write(response.content)

440409

### Reading the MRTS Data and Data Preparation

In [ ]:
xlsx = pd.ExcelFile("mrtssales92-present.xlsx")

sheets_dict = pd.read_excel(xlsx)

sales_df = pd.DataFrame()

for sheet in reversed(xlsx.sheet_names):
    
    # Capture the 65 Adjusted business types from column B in the spreadsheet and place them into a dataframe
    categories = pd.read_excel(xlsx, sheet_name=sheet, header = 3, usecols ="B").iloc[2:67]

    # The first sheet, 2022, has 7 less columns than the others so it has an exception
    if sheet == '2022':
        sales_figs = pd.read_excel(xlsx, sheet_name=sheet, header = 4, usecols ="C:M").iloc[1:66]
    else:
        # Otherwise place the 65 rows of values from columns C through N into a dataframe
        sales_figs = pd.read_excel(xlsx, sheet_name=sheet, header = 4, usecols ="C:N").iloc[1:66]

    # Combine the dataframe containing the business types with the dataframe that holds their associated sales figures.
    reshaped_df = pd.concat([categories.reset_index(drop=True),sales_figs.reset_index(drop=True)], axis=1)
    sales_df = pd.concat([sales_df,reshaped_df.reset_index(drop=True)], axis=1)

In [3]:
xlsx = pd.ExcelFile("mrtssales92-present.xlsx")

sheets_dict = pd.read_excel(xlsx)

asis_sales_df = pd.DataFrame()
adj_sales_df = pd.DataFrame()

for sheet in reversed(xlsx.sheet_names):
    
    # Capture the 65 Adjusted business types from column B in the spreadsheet and place them into a dataframe
    asis_categories = pd.read_excel(xlsx, sheet_name=sheet, header = 3, usecols ="B").iloc[2:67]
    adj_categories = pd.read_excel(xlsx, sheet_name=sheet, header = 3, usecols ="B").iloc[69:107]

    # The first sheet, 2022, has 7 less columns than the others so it has an exception
    if sheet == '2022':
        asis_sales_figs = pd.read_excel(xlsx, sheet_name=sheet, header = 4, usecols ="C:M").iloc[1:66]
        adj_sales_figs = pd.read_excel(xlsx, sheet_name=sheet, header = 4, usecols ="C:M").iloc[68:106]
    else:
        # Otherwise place the 65 rows of values from columns C through N into a dataframe
        asis_sales_figs = pd.read_excel(xlsx, sheet_name=sheet, header = 4, usecols ="C:N").iloc[1:66]
        adj_sales_figs = pd.read_excel(xlsx, sheet_name=sheet, header = 4, usecols ="C:N").iloc[68:106]

    # Combine the dataframe containing the business types with the dataframe that holds their associated sales figures.
    asis_reshaped_df = pd.concat([asis_categories.reset_index(drop=True),asis_sales_figs.reset_index(drop=True)], axis=1)
    adj_reshaped_df = pd.concat([adj_categories.reset_index(drop=True),adj_sales_figs.reset_index(drop=True)], axis=1)
    
    asis_sales_df = pd.concat([asis_sales_df,asis_reshaped_df.reset_index(drop=True)], axis=1)
    adj_sales_df = pd.concat([adj_sales_df,adj_reshaped_df.reset_index(drop=True)], axis=1)

    # asis_sales_df["Adjusted"] = "No"
    # adj_sales_df["Adjusted"] = "Yes"

    sales_df = pd.concat([asis_sales_df.reset_index(drop=True),adj_sales_df.reset_index(drop=True)])

In [4]:
sales_df.shape

(103, 402)

In [5]:
sales_df.head()

,Kind of Business,Jan. 1992,Feb. 1992,Mar. 1992,Apr. 1992,May 1992,Jun. 1992,Jul. 1992,Aug. 1992,Sep. 1992,...,Feb. 2022,Mar. 2022,Apr. 2022,May 2022,Jun. 2022,Jul. 2022,Aug. 2022,Sep. 2022,Oct. 2022,Nov. 2022(p)
0,"Retail and food services sales, total",146376,147079,159336,163669,170068,168663,169890,170364,164617,...,579139,681854,681357,705001,697935,690050,702729,664684,688127,694956
1,Retail sales and food services excl motor vehi...,116565,115862,124200,127587,133608,130274,132076,134928,128734,...,462148,541746,544061,572984,565376,562385,566838,539505,561517,577503
2,Retail sales and food services excl gasoline s...,134277,135499,147064,151226,156619,155236,156016,156539,151477,...,529564,617457,616581,632118,622594,615971,634752,602900,623453,635829
3,Retail sales and food services excl motor vehi...,104466,104282,111928,115144,120159,116847,118202,121103,115594,...,412573,477349,479285,500101,490035,488306,498861,477721,496843,518376
4,"Retail sales, total",130683,131244,142488,147175,152420,151849,152586,152476,148158,...,506400,597077,594426,614474,609933,600746,613416,577370,597349,610784


In [6]:
sales_df = sales_df.drop(columns ='Kind of Business').rename(columns={'Nov. 2022(p)': 'Nov 2022'})
sales_df.head()

,Jan. 1992,Feb. 1992,Mar. 1992,Apr. 1992,May 1992,Jun. 1992,Jul. 1992,Aug. 1992,Sep. 1992,Oct. 1992,...,Feb. 2022,Mar. 2022,Apr. 2022,May 2022,Jun. 2022,Jul. 2022,Aug. 2022,Sep. 2022,Oct. 2022,Nov 2022
0,146376,147079,159336,163669,170068,168663,169890,170364,164617,173655,...,579139,681854,681357,705001,697935,690050,702729,664684,688127,694956
1,116565,115862,124200,127587,133608,130274,132076,134928,128734,136917,...,462148,541746,544061,572984,565376,562385,566838,539505,561517,577503
2,134277,135499,147064,151226,156619,155236,156016,156539,151477,159992,...,529564,617457,616581,632118,622594,615971,634752,602900,623453,635829
3,104466,104282,111928,115144,120159,116847,118202,121103,115594,123254,...,412573,477349,479285,500101,490035,488306,498861,477721,496843,518376
4,130683,131244,142488,147175,152420,151849,152586,152476,148158,155987,...,506400,597077,594426,614474,609933,600746,613416,577370,597349,610784


In [7]:
transposed_df = sales_df.transpose()
transposed_df.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
Jan. 1992,146376,116565,134277,104466,130683,100872,33906,29811,26788,25800,...,14134,5745,3063,2682,4318,6515,2668,1282,17170,NaN
Feb. 1992,147079,115862,135499,104282,131244,100027,35220,31217,28203,27031,...,14472,5730,3109,2621,4308,6110,2632,1177,16990,NaN
Mar. 1992,159336,124200,147064,111928,142488,107352,38731,35136,31684,30195,...,14543,5625,3132,2493,4292,6018,2477,1327,16916,NaN
Apr. 1992,163669,127587,151226,115144,147175,111093,40548,36082,32547,30583,...,14529,5729,3188,2541,4391,6327,2757,1464,16677,NaN
May 1992,170068,133608,156619,120159,152420,115960,42213,36460,32883,31095,...,14634,5711,3212,2499,4356,6550,2874,1491,16602,NaN


In [8]:
asis_categories['Kind of Business'] = asis_categories['Kind of Business'].str.replace("Retail sales and food services","RSFS",regex=True)
asis_categories['Kind of Business'] = asis_categories['Kind of Business'].str.replace("\(|\)|\.|,|'","",regex=True).str.replace("-"," ",regex=True)

adj_categories['Kind of Business'] = adj_categories['Kind of Business'].str.replace("Retail sales and food services","RSFS",regex=True)
adj_categories['Kind of Business'] = adj_categories['Kind of Business'].str.replace("\(|\)|\.|,|'","",regex=True).str.replace("-"," ",regex=True)

transposed_df.columns = pd.concat([asis_categories['Kind of Business'].reset_index(drop=True),adj_categories['Kind of Business'].reset_index(drop=True)]).to_list()
transposed_df = transposed_df.replace(to_replace="\(S\)|\(NA\)",value=np.nan,regex=True)
transposed_df = transposed_df.where((pd.notnull(transposed_df)), 0)

transposed_df.head()

,Retail and food services sales total,RSFS excl motor vehicle and parts,RSFS excl gasoline stations,RSFS excl motor vehicle and parts and gasoline stations,Retail sales total,Retail sales total excl motor vehicle and parts dealers,GAFO1,Motor vehicle and parts dealers,Automobile and other motor vehicle dealers,Automobile dealers,...,Department stores,Other general merchandise stores,Warehouse clubs and superstores,All other gen merchandise stores,Miscellaneous stores retailers,Nonstore retailers,Electronic shopping and mail order houses,Fuel dealers,Food services and drinking places,NaN
Jan. 1992,146376.0,116565.0,134277.0,104466.0,130683.0,100872.0,33906.0,29811.0,26788.0,25800.0,...,14134.0,5745.0,3063.0,2682.0,4318.0,6515.0,2668.0,1282.0,17170.0,0.0
Feb. 1992,147079.0,115862.0,135499.0,104282.0,131244.0,100027.0,35220.0,31217.0,28203.0,27031.0,...,14472.0,5730.0,3109.0,2621.0,4308.0,6110.0,2632.0,1177.0,16990.0,0.0
Mar. 1992,159336.0,124200.0,147064.0,111928.0,142488.0,107352.0,38731.0,35136.0,31684.0,30195.0,...,14543.0,5625.0,3132.0,2493.0,4292.0,6018.0,2477.0,1327.0,16916.0,0.0
Apr. 1992,163669.0,127587.0,151226.0,115144.0,147175.0,111093.0,40548.0,36082.0,32547.0,30583.0,...,14529.0,5729.0,3188.0,2541.0,4391.0,6327.0,2757.0,1464.0,16677.0,0.0
May 1992,170068.0,133608.0,156619.0,120159.0,152420.0,115960.0,42213.0,36460.0,32883.0,31095.0,...,14634.0,5711.0,3212.0,2499.0,4356.0,6550.0,2874.0,1491.0,16602.0,0.0


In [9]:
melted_sales_df = pd.melt(transposed_df, var_name="Kind of Business", value_name="Sales", ignore_index=False)
melted_sales_df.head(10)

,Kind of Business,Sales
Jan. 1992,Retail and food services sales total,146376.0
Feb. 1992,Retail and food services sales total,147079.0
Mar. 1992,Retail and food services sales total,159336.0
Apr. 1992,Retail and food services sales total,163669.0
May 1992,Retail and food services sales total,170068.0
Jun. 1992,Retail and food services sales total,168663.0
Jul. 1992,Retail and food services sales total,169890.0
Aug. 1992,Retail and food services sales total,170364.0
Sep. 1992,Retail and food services sales total,164617.0
Oct. 1992,Retail and food services sales total,173655.0


In [10]:
melted_sales_df.reset_index(inplace=True)
melted_sales_df.head()

,index,Kind of Business,Sales
0,Jan. 1992,Retail and food services sales total,146376.0
1,Feb. 1992,Retail and food services sales total,147079.0
2,Mar. 1992,Retail and food services sales total,159336.0
3,Apr. 1992,Retail and food services sales total,163669.0
4,May 1992,Retail and food services sales total,170068.0


In [11]:
melted_sales_df.rename(columns={'index':'Month'}, inplace=True)
melted_sales_df.head()

,Month,Kind of Business,Sales
0,Jan. 1992,Retail and food services sales total,146376.0
1,Feb. 1992,Retail and food services sales total,147079.0
2,Mar. 1992,Retail and food services sales total,159336.0
3,Apr. 1992,Retail and food services sales total,163669.0
4,May 1992,Retail and food services sales total,170068.0


In [12]:
melted_sales_df['Month']=pd.to_datetime(melted_sales_df['Month'])
melted_sales_df.head()

,Month,Kind of Business,Sales
0,1992-01-01,Retail and food services sales total,146376.0
1,1992-02-01,Retail and food services sales total,147079.0
2,1992-03-01,Retail and food services sales total,159336.0
3,1992-04-01,Retail and food services sales total,163669.0
4,1992-05-01,Retail and food services sales total,170068.0


In [13]:
melted_sales_df.shape

(38213, 3)

### Exploring Percentage Change

In [14]:
# transposed_df = transposed_df.set_index('Month')

percent_changes = transposed_df.pct_change()

percent_changes.head()

# pd.options.display.float_format = '{:.1%}'.format

,Retail and food services sales total,RSFS excl motor vehicle and parts,RSFS excl gasoline stations,RSFS excl motor vehicle and parts and gasoline stations,Retail sales total,Retail sales total excl motor vehicle and parts dealers,GAFO1,Motor vehicle and parts dealers,Automobile and other motor vehicle dealers,Automobile dealers,...,Department stores,Other general merchandise stores,Warehouse clubs and superstores,All other gen merchandise stores,Miscellaneous stores retailers,Nonstore retailers,Electronic shopping and mail order houses,Fuel dealers,Food services and drinking places,NaN
Jan. 1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Feb. 1992,0.004803,-0.006031,0.009101,-0.001761,0.004293,-0.008377,0.038754,0.047164,0.052822,0.047713,...,0.023914,-0.002611,0.015018,-0.022744,-0.002316,-0.062164,-0.013493,-0.081903,-0.010483,NaN
Mar. 1992,0.083336,0.071965,0.085351,0.073320,0.085672,0.073230,0.099688,0.125541,0.123427,0.117051,...,0.004906,-0.018325,0.007398,-0.048836,-0.003714,-0.015057,-0.058891,0.127443,-0.004356,NaN
Apr. 1992,0.027194,0.027271,0.028301,0.028733,0.032894,0.034848,0.046913,0.026924,0.027238,0.012850,...,-0.000963,0.018489,0.017880,0.019254,0.023066,0.051346,0.113040,0.103240,-0.014129,NaN
May 1992,0.039097,0.047191,0.035662,0.043554,0.035638,0.043810,0.041062,0.010476,0.010324,0.016741,...,0.007227,-0.003142,0.007528,-0.016529,-0.007971,0.035246,0.042437,0.018443,-0.004497,NaN


In [15]:
melted_pc_df = pd.melt(percent_changes, var_name="Kind of Business", value_name="Percent Change", ignore_index=False)
melted_pc_df.head(10)

,Kind of Business,Percent Change
Jan. 1992,Retail and food services sales total,NaN
Feb. 1992,Retail and food services sales total,0.004803
Mar. 1992,Retail and food services sales total,0.083336
Apr. 1992,Retail and food services sales total,0.027194
May 1992,Retail and food services sales total,0.039097
Jun. 1992,Retail and food services sales total,-0.008261
Jul. 1992,Retail and food services sales total,0.007275
Aug. 1992,Retail and food services sales total,0.002790
Sep. 1992,Retail and food services sales total,-0.033734
Oct. 1992,Retail and food services sales total,0.054903


In [16]:
melted_pc_df.reset_index(inplace=True)
melted_pc_df.head()

,index,Kind of Business,Percent Change
0,Jan. 1992,Retail and food services sales total,NaN
1,Feb. 1992,Retail and food services sales total,0.004803
2,Mar. 1992,Retail and food services sales total,0.083336
3,Apr. 1992,Retail and food services sales total,0.027194
4,May 1992,Retail and food services sales total,0.039097


In [17]:
melted_pc_df.rename(columns={'index':'Month'}, inplace=True)
melted_pc_df.head()

,Month,Kind of Business,Percent Change
0,Jan. 1992,Retail and food services sales total,NaN
1,Feb. 1992,Retail and food services sales total,0.004803
2,Mar. 1992,Retail and food services sales total,0.083336
3,Apr. 1992,Retail and food services sales total,0.027194
4,May 1992,Retail and food services sales total,0.039097


In [18]:
melted_pc_df['Month']=pd.to_datetime(melted_pc_df['Month'])
melted_pc_df.head()

,Month,Kind of Business,Percent Change
0,1992-01-01,Retail and food services sales total,NaN
1,1992-02-01,Retail and food services sales total,0.004803
2,1992-03-01,Retail and food services sales total,0.083336
3,1992-04-01,Retail and food services sales total,0.027194
4,1992-05-01,Retail and food services sales total,0.039097


In [19]:
melted_pc_df.shape

(38213, 3)

In [20]:
biz_sales = pd.merge(melted_sales_df, melted_pc_df,  how='left', on=['Month','Kind of Business'])
biz_sales.head()

,Month,Kind of Business,Sales,Percent Change
0,1992-01-01,Retail and food services sales total,146376.0,NaN
1,1992-02-01,Retail and food services sales total,147079.0,0.004803
2,1992-03-01,Retail and food services sales total,159336.0,0.083336
3,1992-04-01,Retail and food services sales total,163669.0,0.027194
4,1992-05-01,Retail and food services sales total,170068.0,0.039097


In [21]:
biz_sales.shape

(64925, 4)

In [22]:
biz_sales.to_csv('data/mrtssales92-present.csv')

## Reference

US Census Bureau: Monthly Retail Trade. https://www.census.gov/retail/index.html.